In [1]:
import gensim
import os
import collections
import smart_open
import random
import multiprocessing
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint

In [2]:
#read class label file
#create mapping from id to labels  
#iso-8859-1 , encoding="iso-8859-1"
conceptLabelDict={}
errors=[]

def read_label(fname):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t")
            if len(splitted)==3:
                conceptLabelDict[splitted[1]] = splitted[2].replace("\r\n", "")
            else:
                errors.append(splitted)

label_file = "ontClassLabels.txt"
read_label(label_file)
print(conceptLabelDict["446087008"])
print(conceptLabelDict["132818006"])
print(errors)


montgomery-Ãsberg depression rating scale
Ãsbos chicken breed
[['int 20170731 ']]


In [17]:
print(len(conceptLabelDict))

336831


In [3]:
conceptMappingDict={}

In [18]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t", 1)
            
            line = line.decode("iso-8859-1")
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                conceptMappingDict[i]= int(splitted[0])
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [ ]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f): 
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [randomStr(10),i])

In [ ]:
#gensim.utils.simple_preprocess??
gensim.models.doc2vec.TaggedDocument??

In [19]:
train_file = "ontClassTopology.txt"

train_corpus = list(read_corpus(train_file))

In [20]:
print(len(train_corpus))
train_corpus[11256:11257]

336871


[TaggedDocument(words=['entire', 'body', 'conduit', 'structure', 'of', 'interosseous', 'metatarsal', 'spaces', 'entire', 'interosseous', 'metatarsal', 'spaces', 'entire', 'interosseous', 'metatarsal', 'spaces'], tags=[11256])]

In [ ]:
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=1, iter=5, alpha=0.025, min_alpha=0.025, sample = 1e-3)

In [ ]:
gensim.models.doc2vec.Doc2Vec??

In [ ]:
model.build_vocab(train_corpus)

In [ ]:
len(model.wv.vocab)

In [ ]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.iter)

In [ ]:
# inferred_vector = model.infer_vector(['clinical', 'finding', 'evaluation', 'prevent', 'sampling', 'foot'])
inferred_vector = model.infer_vector(['congenital', 'prolong', 'rupture', 'premature', 'membrane', 'lung'])


sims = model.docvecs.most_similar([inferred_vector], topn=10)
pprint(sims)

In [ ]:
for sim in sims:
    print(train_corpus[sim[0]].words, "score: ", sim[1])
    
for sim in sims:
    print(conceptMappingDict[sim[0]], "score: ", sim[1])
  

In [ ]:
from tempfile import mkstemp

fs, temp_path = mkstemp("gensim_temp")  # creates a temp file

model.save(temp_path)  # save the model

In [ ]:
new_model = gensim.models.Doc2Vec.load(temp_path)

In [21]:
cores = multiprocessing.cpu_count()

print(cores)
models = [
    # PV-DBOW 
    Doc2Vec(dm=0, dbow_words=1, size=200, window=8, min_count=1, iter=10, workers=cores),
    # PV-DM w/average
    Doc2Vec(dm=1, dm_mean=1, size=200, window=8, min_count=1, iter =10, workers=cores),
]


8


In [22]:
models[0].build_vocab(train_corpus)
print(str(models[0]))
models[1].reset_from(models[0])
print(str(models[1]))

Doc2Vec(dbow+w,d200,n5,w8,s0.001,t8)
Doc2Vec(dm/m,d200,n5,w8,s0.001,t8)


In [23]:
for model in models:
    %%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.iter)

CPU times: user 1h 52min 19s, sys: 21.4 s, total: 1h 52min 40s
Wall time: 15min 29s
CPU times: user 21min 36s, sys: 48 s, total: 22min 24s
Wall time: 6min 33s


In [34]:
for i, model in enumerate(models):
    temp_path = "model" + str(i)  # creates a temp file
    model.save(temp_path)

In [37]:
model = Doc2Vec.load("model1") 

pprint(model.docvecs.most_similar([inferred_vector], topn=20))

[(327366, 0.6250582933425903),
 (277506, 0.6003328561782837),
 (323993, 0.599632978439331),
 (155455, 0.5936585664749146),
 (12484, 0.5933293700218201),
 (213464, 0.5608265399932861),
 (14683, 0.5582849979400635),
 (10707, 0.551878035068512),
 (1838, 0.5508819222450256),
 (2076, 0.5499801635742188),
 (65500, 0.5444504618644714),
 (214, 0.5439452528953552),
 (257706, 0.5381863713264465),
 (272969, 0.5356877446174622),
 (336473, 0.5330525636672974),
 (168031, 0.5322983264923096),
 (279970, 0.526287317276001),
 (172394, 0.5259836316108704),
 (294628, 0.5208942890167236),
 (87527, 0.5191961526870728)]


In [26]:
for model in models:
    print(str(model))
    inferred_vector = model.infer_vector(['congenital', 'prolong', 'rupture', 'premature', 'membrane', 'lung'])
    pprint(model.docvecs.most_similar([inferred_vector], topn=20))

Doc2Vec(dbow+w,d200,n5,w8,s0.001,t8)
[(184656, 0.6794406771659851),
 (290617, 0.678496241569519),
 (291604, 0.6688330769538879),
 (59202, 0.6439175009727478),
 (283984, 0.642685055732727),
 (124086, 0.6418383717536926),
 (148659, 0.6411072015762329),
 (32814, 0.63981032371521),
 (106078, 0.6326524615287781),
 (257551, 0.6319582462310791),
 (330504, 0.6302298307418823),
 (54569, 0.6280962228775024),
 (132857, 0.6276460289955139),
 (255937, 0.6274430751800537),
 (20276, 0.6268020868301392),
 (14829, 0.6265949606895447),
 (95129, 0.6262406706809998),
 (285139, 0.6245691776275635),
 (80666, 0.6227687001228333),
 (111487, 0.6225621104240417)]
Doc2Vec(dm/m,d200,n5,w8,s0.001,t8)
[(327366, 0.7474574446678162),
 (65500, 0.6958821415901184),
 (277506, 0.693389892578125),
 (12484, 0.681053876876831),
 (168031, 0.6267901659011841),
 (44817, 0.6190064549446106),
 (243108, 0.6178455948829651),
 (198225, 0.6071273684501648),
 (63152, 0.5938667058944702),
 (68300, 0.5772860646247864),
 (75312, 0.56915

In [30]:
for model in models:
    print(str(model))
    inferred_vector = model.infer_vector(['congenital', 'prolong', 'rupture', 'premature', 'membrane', 'lung'])
    sims= model.docvecs.most_similar([inferred_vector], topn=20)
    for sim in sims:
        index = str(conceptMappingDict[sim[0]])
        print(conceptLabelDict[index], "score: ", sim[1])
    

Doc2Vec(dbow+w,d200,n5,w8,s0.001,t8)
congenital pulmonary hypoplasia due to prolonged premature rupture of membranes score:  0.6569488644599915
prolonged premature rupture of membranes score:  0.64715576171875
congenital pulmonary hypoplasia due to lung space occupying lesion score:  0.6402868628501892
congenital tracheocele score:  0.6391810178756714
prolonged spontaneous rupture of membranes score:  0.6297885179519653
preterm premature rupture of membranes score:  0.625205397605896
ruptured membranes score:  0.6226955652236938
preterm premature rupture of membranes with onset of labor unknown score:  0.6188146471977234
membrane rupture with delivery delay score:  0.6165027022361755
congenital pulmonary acinar dysplasia score:  0.6122896671295166
spontaneous hindwater rupture of membranes score:  0.6121758222579956
forewater rupture of amniotic membrane score:  0.6119036674499512
premature rupture of membranes with antenatal problem score:  0.6112538576126099
nontraumatic gastric rupt

In [ ]:
for model in models:
    print(str(model))
    vec = [model.docvecs["Lady Gaga"] - model["american"] + model["japanese"]]
    pprint([m for m in model.docvecs.most_similar(vec, topn=11) if m[0] != "Lady Gaga"])

In [ ]:
print("Total: ",len(train_corpus))


ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    print("Processing: ", doc_id)
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

In [ ]:
import random, string

def randomStr(length):
   letters = string.ascii_lowercase
   return ''.join(random.choice(letters) for i in range(length))


In [ ]:
collections.Counter(ranks) 